In [1]:
import pandas as pd
import os
import string
import re

In [2]:
#Adjust jupyter notebook margins
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [24]:
#Read 18000 job postings where 800 are fake
fake_jobs_pd = pd.read_csv("data_source/fake_job_postings.csv")

In [25]:
#have a glimpse of the file
fake_jobs_pd.head()

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [26]:
#flattening the location column into 3 different columns for country, state, and city.
ci_st_co=fake_jobs_pd['location'].tolist()

print(type(ci_st_co))
cou=[]
sta=[]
cit=[]


for i in ci_st_co:
        #print(i)
    try:
        co=i.split(', ')
    
        cou.append(co[0])
        sta.append(co[1])
        cit.append(co[2])
        
    except:
        
        cou.append('')
        sta.append('')
        cit.append('')
print(len(cou))

<class 'list'>
17974


In [27]:
# while loop to get arrays the same size.
print(len(cou))
print(len(sta))
print(len(cit))
k=len(cou)
while k>len(cit):
    
    del cou[-1]
    k=k-1
    print(len(cou))

17974
17880
17880
17973
17972
17971
17970
17969
17968
17967
17966
17965
17964
17963
17962
17961
17960
17959
17958
17957
17956
17955
17954
17953
17952
17951
17950
17949
17948
17947
17946
17945
17944
17943
17942
17941
17940
17939
17938
17937
17936
17935
17934
17933
17932
17931
17930
17929
17928
17927
17926
17925
17924
17923
17922
17921
17920
17919
17918
17917
17916
17915
17914
17913
17912
17911
17910
17909
17908
17907
17906
17905
17904
17903
17902
17901
17900
17899
17898
17897
17896
17895
17894
17893
17892
17891
17890
17889
17888
17887
17886
17885
17884
17883
17882
17881
17880


In [28]:
#new data frame for flattened locations.
datas={'country':cou,
       'state':sta,
       'city':cit }   
uplocs=pd.DataFrame(datas)
print(len(datas['country']))
uplocs.head()

17880


,country,state,city
0,US,NY,New York
1,NZ,,Auckland
2,US,IA,Wever
3,US,DC,Washington
4,US,FL,Fort Worth


In [29]:
#drop the non useful columns in fake job dataset
fake_job_drops=['salary_range','company_profile','benefits','telecommuting',\
               'has_company_logo','has_questions']

fake_jobs_pd
#try:
for cols in fake_job_drops:
    fake_jobs_pd=fake_jobs_pd.drop(f'{cols}',1)
    #updated_fake_jobs_pd.head()
#except:
        #updated_ds_jobs_pd.head()
print(len(fake_jobs_pd))
fake_jobs_pd.head()

17880


,job_id,title,location,department,description,requirements,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,"Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [9]:
fake_jobs_pd.head()

,job_id,title,location,department,description,requirements,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,"Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [10]:
#technician = re.compile(r'technician', re.I)
#developer = re.compile(r'developer', re.I)
#engineer = re.compile(r'engineer', re.I)
#programmer = re.compile(r'programmer', re.I)
#analyst = re.compile(r'analyst', re.I)

In [11]:
#words_to_filter = ['Technician', 'Developer', 'Engineer','Programmer','Analyst']
#test = fake_jobs_pd.query("{0} is not None or {1} is not None or {2} is not None or {3} is not None or {4} is not None".format(
    #technician.search(title),developer.search(title),engineer.search(title),programmer.search(title),analyst.search(title)), engine='python')
#test

In [12]:
#filter out for techy jobs.

words_to_filter = ['Technician', 'Developer', 'Engineer','Programmer','Analyst']
job_ids=[]
for job_name in fake_jobs_pd['title'].tolist():
    for names in words_to_filter:
        if re.search(names,job_name):
            fake_jobs_pd.loc[fake_jobs_pd['title']==job_name,"Tech Job Check"] =True
        break
        
fake_jobs_pd.head()

,job_id,title,location,department,description,requirements,employment_type,required_experience,required_education,industry,function,fraudulent,Tech Job Check
0,1,Marketing Intern,"US, NY, New York",Marketing,"Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,Other,Internship,NaN,NaN,Marketing,0,NaN
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0,NaN
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,NaN,NaN,NaN,NaN,0,NaN
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0,NaN
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0,NaN


In [30]:
cleanfakes=fake_jobs_pd.join(uplocs)
cleanfakes.to_csv('cleaned fake jobs.csv')

In [31]:
#Get only Fraud records
if fake_jobs_pd.index.name != 'fraudulent':
   fake_jobs_pd.set_index("fraudulent", inplace = True)
fake_jobs = fake_jobs_pd.loc[1]

In [32]:
#Create dict to collect word count of fake job postings
word_count = {}
#Count words in fake job postings
for description in fake_jobs['description']:
 for word in str(description).split():
    if word_count.get(word, 0) != 0:
         word_count[word] += 1  
    else:
         word_count[word] = 1

#Create dataframe
data_items = sorted(word_count.items(), key=lambda kv: kv[1], reverse=True)
data_list = list(data_items)
word_count_pd = pd.DataFrame(data_list,columns=["word","counter"])

#Limit size of words to analyze
word_count_pd.drop(word_count_pd[word_count_pd['word'][:].str.len() < 4].index, inplace = True)

#drop prepositions and other not significant words
word_count_pd.drop(word_count_pd[word_count_pd['word'] == 'with'].index, inplace = True) 
word_count_pd.drop(word_count_pd[word_count_pd['word'] == 'that'].index, inplace = True)
word_count_pd.drop(word_count_pd[word_count_pd['word'] == 'from'].index, inplace = True)
word_count_pd.drop(word_count_pd[word_count_pd['word'] == 'have'].index, inplace = True)
word_count_pd.drop(word_count_pd[word_count_pd['word'] == 'your'].index, inplace = True)
word_count_pd.drop(word_count_pd[word_count_pd['word'] == 'this'].index, inplace = True)
word_count_pd.drop(word_count_pd[word_count_pd['word'] == 'their'].index, inplace = True)
word_count_pd.drop(word_count_pd[word_count_pd['word'] == '&amp;'].index, inplace = True)
word_count_pd.drop(word_count_pd[word_count_pd['word'] == 'will'].index, inplace = True)
word_count_pd.drop(word_count_pd[word_count_pd['word'] == 'This'].index, inplace = True)
word_count_pd.drop(word_count_pd[word_count_pd['word'] == 'more'].index, inplace = True)
word_count_pd.drop(word_count_pd[word_count_pd['word'] == 'into'].index, inplace = True)
word_count_pd.drop(word_count_pd[word_count_pd['word'] == 'also'].index, inplace = True)



In [33]:
#Get only Real Posting records
if fake_jobs_pd.index.name != 'fraudulent':
   fake_jobs_pd.set_index("fraudulent", inplace = True)
real_jobs_pd = fake_jobs_pd.loc[0]


In [34]:
#Create dict to collect word count for real job postings
word_count_real = {}

#Count words in fake job postings
for description in real_jobs_pd['description']:
 for word in str(description).split():
    if word_count_real.get(word, 0) != 0:
         word_count_real[word] += 1  
    else:
         word_count_real[word] = 1

#Create dataframe
data_items = sorted(word_count_real.items(), key=lambda kv: kv[1], reverse=True)
data_list = list(data_items)
word_count_real_pd = pd.DataFrame(data_list,columns=["word","counter"])

#Limit size of words to analyze
word_count_real_pd.drop(word_count_real_pd[word_count_real_pd['word'][:].str.len() < 4].index, inplace = True)

#drop prepositions and other not significant words
word_count_real_pd.drop(word_count_real_pd[word_count_real_pd['word'] == 'with'].index, inplace = True) 
word_count_real_pd.drop(word_count_real_pd[word_count_real_pd['word'] == 'that'].index, inplace = True)
word_count_real_pd.drop(word_count_real_pd[word_count_real_pd['word'] == 'from'].index, inplace = True)
word_count_real_pd.drop(word_count_real_pd[word_count_real_pd['word'] == 'have'].index, inplace = True)
word_count_real_pd.drop(word_count_real_pd[word_count_real_pd['word'] == 'your'].index, inplace = True)
word_count_real_pd.drop(word_count_real_pd[word_count_real_pd['word'] == 'this'].index, inplace = True)
word_count_real_pd.drop(word_count_real_pd[word_count_real_pd['word'] == 'their'].index, inplace = True)
word_count_real_pd.drop(word_count_real_pd[word_count_real_pd['word'] == '&amp;'].index, inplace = True)
word_count_real_pd.drop(word_count_real_pd[word_count_real_pd['word'] == 'will'].index, inplace = True)
word_count_real_pd.drop(word_count_real_pd[word_count_real_pd['word'] == 'This'].index, inplace = True)
word_count_real_pd.drop(word_count_real_pd[word_count_real_pd['word'] == 'more'].index, inplace = True)
word_count_real_pd.drop(word_count_real_pd[word_count_real_pd['word'] == 'into'].index, inplace = True)
word_count_real_pd.drop(word_count_real_pd[word_count_real_pd['word'] == 'also'].index, inplace = True)

In [35]:
#Create dataframe of words that are only in the fake postings (by comparing with words in real postings)
fake_words_to_check = word_count_pd.merge(word_count_real_pd, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='left_only']

#Limit the number of fake words to 50
fake_words_to_check= fake_words_to_check[0:50]

#Add weight to each fake word (average)
fake_words_to_check['weight']=fake_words_to_check['counter']/fake_words_to_check['counter'].sum()

#Add index to df
fake_words_to_check.set_index("word", inplace = True)

#Generate CSV file of count of words from fake job postings
word_count_pd.to_csv("fake_words_to_check.csv")


In [36]:
#Read 10000 Data Scientist job postings
ds_jobs_pd = pd.read_csv("data_source/data_scientist_united_states_job_postings_jobspikr.csv")

In [37]:
ds_jobs_pd=ds_jobs_pd.rename_axis('ds_key')
ds_jobs_pd.head()

,crawl_timestamp,url,job_title,category,company_name,city,state,country,inferred_city,inferred_state,...,job_description,job_type,salary_offered,job_board,geo,cursor,contact_email,contact_phone_number,uniq_id,html_job_description
ds_key,,,,,,,,,,,,,,,,,,,,,
0,2019-02-06 05:26:22 +0000,https://www.indeed.com/viewjob?jk=fd83355c2b23...,Enterprise Data Scientist I,Accounting/Finance,Farmers Insurance Group,Woodland Hills,CA,Usa,Woodland hills,California,...,Read what people are saying about working here...,Undefined,NaN,indeed,usa,1549432819114777,NaN,NaN,3b6c6acfcba6135a31c83bd7ea493b18,NaN
1,2019-02-06 05:33:41 +0000,https://www.dice.com/jobs/detail/Data-Scientis...,Data Scientist,NaN,Luxoft USA Inc,Middletown,NJ,Usa,Middletown,New jersey,...,We have an immediate opening for a Sharp Data ...,Undefined,NaN,dice,usa,1549432819122106,NaN,NaN,741727428839ae7ada852eebef29b0fe,NaN
2,2019-02-06 05:33:35 +0000,https://www.dice.com/jobs/detail/Data-Scientis...,Data Scientist,NaN,Cincinnati Bell Technology Solutions,New York,NY,Usa,New york,New york,...,Candidates should have the following backgroun...,Full Time,NaN,dice,usa,1549432819236156,NaN,NaN,cdc9ef9a1de327ccdc19cc0d07dbbb37,NaN
3,2019-02-06 05:33:42 +0000,https://www.indeed.com/viewjob?jk=841edd86ead2...,"Data Scientist, Aladdin Wealth Tech, Associate...",Accounting/Finance,BlackRock,New York,NY 10055 (Midtown area),Usa,New york,New york,...,Read what people are saying about working here...,Undefined,NaN,indeed,usa,1549432819259473,NaN,NaN,1c8541cd2c2c924f9391c7d3f526f64e,NaN
4,2019-02-06 05:48:23 +0000,https://job-openings.monster.com/senior-data-s...,Senior Data Scientist,biotech,CyberCoders,Charlotte,NC,Usa,Charlotte,North carolina,...,We are seeking an extraordinary Data Scientist...,Full Time,NaN,monster,usa,1549436429015957,NaN,NaN,445652a560a5441060857853cf267470,NaN


In [38]:
drops=['crawl_timestamp','url','company_name','inferred_city','inferred_state','inferred_country',\
    'salary_offered','job_board','geo','cursor','contact_email','contact_phone_number',\
       'uniq_id','html_job_description','post_date']

updated_ds_jobs_pd=ds_jobs_pd
#try:
for item in drops:
    ds_jobs_pd=ds_jobs_pd.drop(f'{item}',1)
    ds_jobs_pd.head()
#except:
        #updated_ds_jobs_pd.head()
ds_jobs_pd.to_csv("cleaned data science jobs.csv")
ds_jobs_pd.head()

,job_title,category,city,state,country,job_description,job_type
ds_key,,,,,,,
0,Enterprise Data Scientist I,Accounting/Finance,Woodland Hills,CA,Usa,Read what people are saying about working here...,Undefined
1,Data Scientist,NaN,Middletown,NJ,Usa,We have an immediate opening for a Sharp Data ...,Undefined
2,Data Scientist,NaN,New York,NY,Usa,Candidates should have the following backgroun...,Full Time
3,"Data Scientist, Aladdin Wealth Tech, Associate...",Accounting/Finance,New York,NY 10055 (Midtown area),Usa,Read what people are saying about working here...,Undefined
4,Senior Data Scientist,biotech,Charlotte,NC,Usa,We are seeking an extraordinary Data Scientist...,Full Time


In [41]:
#Identify possible fake records in the 10000 Data Scientist job postings

#used just for test
ds_jobs_pdx = pd.read_csv("data_source/data_scientist_united_states_job_postings_jobspikr.csv")

lines = len(ds_jobs_pdx)
fake_ds_job_posting = pd.DataFrame()

#Loop thru individual records of DS job postings
for i in range(0,lines):
   ds_jobs_pd = ds_jobs_pdx.iloc[i]
   #Initiate dictionary that will sum the weight of the fake words
   fake_rate = {}
   words_count = 0

   for words in str(ds_jobs_pd['job_description']).split():
       words_count += 1
       
       if words in fake_words_to_check.index:
          idx = fake_words_to_check['weight'].where(fake_words_to_check.index == words).dropna()          
          if fake_rate.get(words, 0) != 0:
             #print(f"entrei no achei#", words, "#before", fake_rate)
             fake_rate[words] = fake_rate[words] + idx[0]
             #print(f"entrei no achei#", words, "#after", fake_rate)
          else:
            fake_rate.update({words: idx[0] })

   #Calc % of fake of the record based on rate >=1.3
   if sum(fake_rate.values()) >= 1.3:
        fake_ds_job_posting = fake_ds_job_posting.append(ds_jobs_pd)
        
         
#Generate CSV file of possible Data Scientist fake job postings
word_count_pd.to_csv("fake_words_to_check.csv")